<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pyLDAvis


import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [ ]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(1)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57


In [ ]:
%%time

bug_report = bug_report.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 2]

bug_report.head(1)

CPU times: user 76.3 ms, sys: 3.48 ms, total: 79.8 ms
Wall time: 81.9 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [ ]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 490 ms, sys: 0 ns, total: 490 ms
Wall time: 492 ms


In [ ]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

CPU times: user 15.7 s, sys: 0 ns, total: 15.7 s
Wall time: 15.7 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [ ]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.loc[[0]]

CPU times: user 26.1 s, sys: 74.5 ms, total: 26.1 s
Wall time: 26.2 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [ ]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

CPU times: user 9.43 ms, sys: 829 µs, total: 10.3 ms
Wall time: 13.6 ms


,Issue_id,Duplicated_issue,Title,Description
0,25,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful, pulldown, connection, wizard, default]"


In [ ]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(1)

CPU times: user 18.7 ms, sys: 2.7 ms, total: 21.4 ms
Wall time: 23 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [ ]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 500:
      break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 correctly
81 delete
82 deletion
83 distinction
84 especially
85 focus
86 indicate
87 indication
88 indicator
8

In [ ]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]
bag_of_words_corpus[0]

CPU times: user 1.36 s, sys: 134 ms, total: 1.5 s
Wall time: 1.51 s


[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1)]

In [ ]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. "actually" word seen 1 time.
2. "builders" word seen 2 time.
3. "change" word seen 5 time.
4. "comment" word seen 1 time.
5. "compare" word seen 1 time.
6. "complete" word seen 1 time.
7. "consider" word seen 1 time.
8. "consistency" word seen 1 time.
9. "contain" word seen 1 time.
10. "default" word seen 1 time.
11. "derive" word seen 1 time.
12. "document" word seen 1 time.
13. "drive" word seen 1 time.
14. "editor" word seen 3 time.
15. "effect" word seen 1 time.
16. "ensure" word seen 1 time.
17. "external" word seen 3 time.
18. "extremely" word seen 1 time.
19. "fail" word seen 1 time.
20. "inconsistent" word seen 1 time.
21. "internal" word seen 1 time.
22. "maintain" word seen 2 time.
23. "mandatory" word seen 1 time.
24. "miss" word seen 1 time.
25. "modify" word seen 1 time.
26. "move" word seen 1 time.
27. "navigator" word seen 2 time.
28. "network" word seen 1 time.
29. "occur" word seen 1 time.
30. "open" word seen 1 time.
31. "optimal" word seen 1 time.
32. "problem" wo

In [ ]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

CPU times: user 6min 24s, sys: 24.1 s, total: 6min 49s
Wall time: 10min 52s


In [ ]:
%%time

for id, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(id, topic))

Topic: 0 
Words: 0.066*"thread" + 0.042*"javathread" + 0.039*"browser" + 0.035*"threadblocked" + 0.032*"daemon" + 0.023*"breakpoint" + 0.023*"progress" + 0.021*"monitor" + 0.021*"native" + 0.020*"memory"
Topic: 1 
Words: 0.060*"dialog" + 0.047*"select" + 0.034*"search" + 0.034*"button" + 0.021*"result" + 0.019*"character" + 0.017*"display" + 0.013*"reproduce" + 0.012*"preference" + 0.012*"preferences"
Topic: 2 
Words: 0.101*"eclipse" + 0.035*"reproduce" + 0.035*"create" + 0.030*"problem" + 0.027*"attach" + 0.027*"windows" + 0.024*"attachment" + 0.020*"application" + 0.015*"run" + 0.014*"information"
Topic: 3 
Words: 0.123*"project" + 0.039*"create" + 0.028*"workspace" + 0.023*"package" + 0.023*"select" + 0.022*"delete" + 0.021*"target" + 0.021*"folder" + 0.018*"resource" + 0.016*"import"
Topic: 4 
Words: 0.117*"source" + 0.093*"message" + 0.090*"method" + 0.039*"exception" + 0.033*"bundle" + 0.022*"occur" + 0.021*"warn" + 0.019*"orgeclipseosgi" + 0.018*"fail" + 0.018*"arguments"
Topic:

In [ ]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))


Perplexity:  -6.551951905573658
CPU times: user 26 s, sys: 597 ms, total: 26.6 s
Wall time: 25.9 s


In [ ]:
%%time

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.625890523104141
CPU times: user 11.6 s, sys: 59.4 ms, total: 11.6 s
Wall time: 11.6 s


In [ ]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


CPU times: user 16.2 s, sys: 601 ms, total: 16.8 s
Wall time: 19.4 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.065211  0.055599       1        1  14.138053
3     -0.146196 -0.010137       2        1  11.449826
2     -0.066319 -0.060426       3        1  10.984910
6     -0.030152 -0.219604       4        1  10.682331
1     -0.159637  0.152231       5        1  10.374827
7     -0.183019  0.111376       6        1  10.318490
9     -0.138821 -0.092231       7        1  10.291874
4      0.252644 -0.185980       8        1   8.762333
8      0.278014  0.306641       9        1   7.728767
0      0.258696 -0.057469      10        1   5.268588, topic_info=                 Term          Freq         Total Category  logprob  loglift
32            project  22570.000000  22570.000000  Default  30.0000  30.0000
372            source  16526.000000  16526.000000  Default  29.0000  29.0000
103           eclipse  28273.000000  28273.000000  Default  28.0000  28.0000
589           message  13116.000000  13116.000000  Default  27.0000  27.0000
13             editor  14026.000000  14026.000000  Default  26.0000  26.0000
116            method  20095.000000  20095.000000  Default  25.0000  25.0000
360            public   9864.000000   9864.000000  Default  24.0000  24.0000
761            import  10841.000000  10841.000000  Default  23.0000  23.0000
101            dialog  11190.000000  11190.000000  Default  22.0000  22.0000
1341           thread   8158.000000   8158.000000  Default  21.0000  21.0000
109            select  14935.000000  14935.000000  Default  20.0000  20.0000
2              change  16413.000000  16413.000000  Default  19.0000  19.0000
358            plugin   8860.000000   8860.000000  Default  18.0000  18.0000
52            display  10697.000000  10697.000000  Default  17.0000  17.0000
130           feature   7013.000000   7013.000000  Default  16.0000  16.0000
248            update   9955.000000   9955.000000  Default  15.0000  15.0000
219            create  19282.000000  19282.000000  Default  14.0000  14.0000
448            return   7904.000000   7904.000000  Default  13.0000  13.0000
365            button   9271.000000   9271.000000  Default  12.0000  12.0000
608            search   5936.000000   5936.000000  Default  11.0000  11.0000
806         exception   6587.000000   6587.000000  Default  10.0000  10.0000
626       perspective   5219.000000   5219.000000  Default   9.0000   9.0000
484            bundle   6285.000000   6285.000000  Default   8.0000   8.0000
4365       javathread   3517.000000   3517.000000  Default   7.0000   7.0000
466            action   5497.000000   5497.000000  Default   6.0000   6.0000
609            window   7021.000000   7021.000000  Default   5.0000   5.0000
525            attach   5466.000000   5466.000000  Default   4.0000   4.0000
698           browser   3325.000000   3325.000000  Default   3.0000   3.0000
4346      libjvmdylib   3637.000000   3637.000000  Default   2.0000   2.0000
2624       attachment   4484.000000   4484.000000  Default   1.0000   1.0000
135         implement   2266.968657   2267.879307   Topic1  -4.6076   1.9559
211    implementation   1701.322081   1702.232721   Topic1  -4.8947   1.9558
948           javadoc   1613.952146   1614.862796   Topic1  -4.9474   1.9557
117           methods   1441.073495   1441.984117   Topic1  -5.0607   1.9557
424           clients    800.996686    801.907301   Topic1  -5.6480   1.9552
2433        deprecate    687.253099    688.163721   Topic1  -5.8011   1.9550
235         listeners    636.156743    637.067357   Topic1  -5.8784   1.9549
1680       instanceof    519.523810    520.434419   Topic1  -6.0809   1.9545
233            listen    386.683509    387.594151   Topic1  -6.3762   1.9539
3046   controlexample    366.483217    367.393952   Topic1  -6.4299   1.9538
1428      tableviewer    353.140355    354.050969   Topic1  -6.4670   1.9537
967          subclass    318.240493    319.151074   Topic1  -6.5710   1.953

In [ ]:
%%time

for i in range(1):
    exec('topic_{} = pd.DataFrame()'.format(i))
    
    for j in range(len(master_reports)):

        topic = lda_model[dictionary.doc2bow(master_reports.Description[j])]
        topic = np.asarray(topic)

        if int(topic[np.argmax(topic[:,1]),0]) == i:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(i,i))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(i,i))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(i,i))